In [1]:
import marisa_trie
import boto3
import taxoniq
import re
import pickle
import sys
import pandas as pd
import numpy as np

Read in CSV file of all data with rank information, then clean data and group it. 

Overall cleaning steps:
- sum up accession counts and total lengths for 'other sequences' (entries with no superkingdom info), drop those rows, then add one row with the cumulative counts for 'other sequences'
- fill in missing kingdom values with superkingdom values for bacteria and archaea
- fill missing values with "unknown_{rank}"
- group data by each rank down to desired level 

In [2]:
all_data = pd.read_csv('test.csv')

In [3]:
all_data

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
0,Eukaryota,Metazoa,Chordata,Ascidiacea,Stolidobranchia,Styelidae,Styela,Styela clava,7725,82293180,30498
1,Eukaryota,Metazoa,Arthropoda,Insecta,Hemiptera,Delphacidae,Nilaparvata,Nilaparvata lugens,108931,111856641,39851
2,Eukaryota,Metazoa,Arthropoda,Insecta,Hymenoptera,Formicidae,Solenopsis,Solenopsis invicta,13686,127796499,34709
3,Eukaryota,Metazoa,Chordata,Mammalia,Primates,Cebidae,Saimiri,Saimiri boliviensis,39432,154142035,53060
4,Eukaryota,Metazoa,Chordata,NaN,Testudines,Geoemydidae,Mauremys,Mauremys reevesii,260615,263305466,86524
...,...,...,...,...,...,...,...,...,...,...,...
1814807,Viruses,Heunggongvirae,Uroviricota,Caudoviricetes,Caudovirales,Herelleviridae,Okubovirus,Bacillus phage H1,10765,100,1
1814808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,midivariant sequence,31896,224,1
1814809,Viruses,Heunggongvirae,Uroviricota,Caudoviricetes,Caudovirales,Siphoviridae,Lambdavirus,Corynephage omega,10714,1905,1
1814810,Viruses,Orthornavirae,Negarnaviricota,Insthoviricetes,Articulavirales,Orthomyxoviridae,Alphainfluenzavirus,Influenza A virus,79695,291,1


In [4]:
ranks = ['superkingdom', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']


In [5]:
all_data.groupby('superkingdom').count()

,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
superkingdom,,,,,,,,,,
Archaea,0,7980,5843,5872,5328,4982,9669,9672,9672,9672
Bacteria,0,342220,317533,323491,316300,305523,397543,397573,397573,397573
Eukaryota,1132826,1153781,1146489,1136876,1111078,913150,1175137,1175177,1175177,1175177
Viruses,177994,177897,177925,177888,178509,163846,185133,185149,185149,185149


Explore NaN values in data.

In [6]:
other_sequences = all_data[all_data.superkingdom.isna()].sort_values(by='num_accessions', ascending=False) # 'other sequences', not actual species
other_sequences

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
37860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uncultured organism,155900,314401643,267990
1534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,synthetic construct,32630,237367640,217078
26922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uncultured prokaryote,198431,65845061,170392
3925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unidentified,32644,50756966,114244
52731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uncultured microorganism,358574,37313762,86083
...,...,...,...,...,...,...,...,...,...,...,...
707267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2928127,328,1
707268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2928128,328,1
707269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2928129,328,1
707270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2928130,328,1


Sum up the length and accessions for the 'other_sequences', drop the rows, and add a new row aggregating them.

In [7]:
length_sum = other_sequences.total_length.sum()
accession_sum = other_sequences.num_accessions.sum()

In [8]:
all_data.dropna(subset='superkingdom', inplace=True)
data = ["other_sequences"] * len(ranks) + [-1, length_sum, accession_sum]
df2 = pd.DataFrame([data], columns = all_data.columns)
all_data = pd.concat([df2, all_data], ignore_index=True)
all_data

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
0,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,-1,3980308547,1140428
1,Eukaryota,Metazoa,Chordata,Ascidiacea,Stolidobranchia,Styelidae,Styela,Styela clava,7725,82293180,30498
2,Eukaryota,Metazoa,Arthropoda,Insecta,Hemiptera,Delphacidae,Nilaparvata,Nilaparvata lugens,108931,111856641,39851
3,Eukaryota,Metazoa,Arthropoda,Insecta,Hymenoptera,Formicidae,Solenopsis,Solenopsis invicta,13686,127796499,34709
4,Eukaryota,Metazoa,Chordata,Mammalia,Primates,Cebidae,Saimiri,Saimiri boliviensis,39432,154142035,53060
...,...,...,...,...,...,...,...,...,...,...,...
1767567,Viruses,Shotokuvirae,Cossaviricota,Quintoviricetes,Piccovirales,Parvoviridae,Protoparvovirus,Rodent protoparvovirus 1,10800,127,1
1767568,Viruses,Heunggongvirae,Uroviricota,Caudoviricetes,Caudovirales,Herelleviridae,Okubovirus,Bacillus phage H1,10765,100,1
1767569,Viruses,Heunggongvirae,Uroviricota,Caudoviricetes,Caudovirales,Siphoviridae,Lambdavirus,Corynephage omega,10714,1905,1
1767570,Viruses,Orthornavirae,Negarnaviricota,Insthoviricetes,Articulavirales,Orthomyxoviridae,Alphainfluenzavirus,Influenza A virus,79695,291,1


Bacteria and Archaea are missing kingdoms. Fill in the kingdom level with the superkingdom.

In [9]:
all_data.loc[all_data.superkingdom == 'Archaea', 'kingdom']= all_data[all_data.superkingdom == 'Archaea'].kingdom.fillna(all_data.superkingdom)
all_data.loc[all_data.superkingdom == 'Bacteria', 'kingdom'] = all_data[all_data.superkingdom == 'Bacteria'].kingdom.fillna(all_data.superkingdom)
all_data[(all_data.superkingdom == 'Archaea') | (all_data.superkingdom == 'Bacteria')]

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
1434,Bacteria,Bacteria,Actinobacteria,Actinomycetia,Corynebacteriales,Mycobacteriaceae,Mycobacterium,Mycobacterium leprae,1769,9807686,403
1450,Bacteria,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus thuringiensis,1428,347410425,7821
1452,Bacteria,Bacteria,Actinobacteria,Actinomycetia,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces lividans,1916,25846403,225
1453,Bacteria,Bacteria,Actinobacteria,Actinomycetia,Streptomycetales,Streptomycetaceae,Streptomyces,Streptomyces coelicolor,1902,18096169,228
1454,Bacteria,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pyogenes,1314,443844907,4159
...,...,...,...,...,...,...,...,...,...,...,...
1767536,Bacteria,Bacteria,Proteobacteria,Alphaproteobacteria,Rickettsiales,Anaplasmataceae,Wolbachia,NaN,953,207,1
1767545,Bacteria,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Salmonella,NaN,590,205,1
1767546,Bacteria,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,Sphingomonas sp. WI4,106806,271,1
1767559,Bacteria,Bacteria,Cyanobacteria,NaN,Synechococcales,Synechococcaceae,Synechococcus,NaN,1129,1886,6


Look at groupings by superkingdom again.

In [10]:
all_data.groupby('superkingdom').count()

,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
superkingdom,,,,,,,,,,
Archaea,9672,7980,5843,5872,5328,4982,9669,9672,9672,9672
Bacteria,397573,342220,317533,323491,316300,305523,397543,397573,397573,397573
Eukaryota,1132826,1153781,1146489,1136876,1111078,913150,1175137,1175177,1175177,1175177
Viruses,177994,177897,177925,177888,178509,163846,185133,185149,185149,185149
other_sequences,1,1,1,1,1,1,1,1,1,1


In [11]:
#what percentage of rows are NA?
(all_data.shape[0] - all_data.dropna().shape[0])/all_data.shape[0]

0.2592986311165825

In [12]:
all_data[all_data["kingdom"] =="Eukaryota"]

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions


In [13]:
# backfill missing ranks to get rid of NAs
# all data has superkingdom at least
def backfill_data(ranks, df): 
  for rank in ranks:
    df[rank] = df[rank].fillna(f'unknown_{rank}')

  return df

all_data = backfill_data(ranks, all_data)


In [14]:
all_data[0:20]

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
0,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,other_sequences,-1,3980308547,1140428
1,Eukaryota,Metazoa,Chordata,Ascidiacea,Stolidobranchia,Styelidae,Styela,Styela clava,7725,82293180,30498
2,Eukaryota,Metazoa,Arthropoda,Insecta,Hemiptera,Delphacidae,Nilaparvata,Nilaparvata lugens,108931,111856641,39851
3,Eukaryota,Metazoa,Arthropoda,Insecta,Hymenoptera,Formicidae,Solenopsis,Solenopsis invicta,13686,127796499,34709
4,Eukaryota,Metazoa,Chordata,Mammalia,Primates,Cebidae,Saimiri,Saimiri boliviensis,39432,154142035,53060
5,Eukaryota,Metazoa,Chordata,unknown_class,Testudines,Geoemydidae,Mauremys,Mauremys reevesii,260615,263305466,86524
6,Eukaryota,Metazoa,Chordata,Aves,Passeriformes,Corvidae,Corvus,Corvus cornix,932674,187689789,42758
7,Eukaryota,Metazoa,Arthropoda,Insecta,Diptera,Culicidae,Culex,Culex pipiens,42434,76102917,28778
8,Eukaryota,Metazoa,Chordata,Actinopteri,Cichliformes,Cichlidae,Oreochromis,Oreochromis aureus,47969,182873946,57938
9,Eukaryota,Metazoa,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Pararge,Pararge aegeria,116150,580760471,22221


Group data by ranks up to desired level of granularity.

In [15]:
groupby_col = 'order'
df_grouped = all_data.groupby(ranks[0: ranks.index(groupby_col)+1], as_index=False).agg(
  {
    'family': 'unique',
    'genus': 'unique',
    'species': 'unique',
    'taxon_id': 'unique',
    'total_length': 'sum', 
    'num_accessions': 'sum'
  })

df_grouped.to_csv('all_data_order_grouped.csv')

In [16]:
df_grouped

,superkingdom,kingdom,phylum,class,order,family,genus,species,taxon_id,total_length,num_accessions
0,Archaea,Archaea,Candidatus Aenigmarchaeota,unknown_class,unknown_order,[unknown_family],"[Candidatus Aenigmarchaeum, unknown_genus]","[uncultured Candidatus Aenigmarchaeum sp., unc...","[1896161, 1462426, 2093792]",3748249,20
1,Archaea,Archaea,Candidatus Bathyarchaeota,unknown_class,unknown_order,[unknown_family],[unknown_genus],"[uncultured miscellaneous Crenarchaeota group,...","[1368239, 1739975, 2026714, 907719, 907717, 90...",34576843,217
2,Archaea,Archaea,Candidatus Diapherotrites,unknown_class,unknown_order,[unknown_family],"[unknown_genus, Candidatus Forterrea]","[uncultured Diapherotrites archaeon, Candidatu...","[1462420, 2107591, 2026736]",1913635,4
3,Archaea,Archaea,Candidatus Geoarchaeota,unknown_class,unknown_order,[unknown_family],[unknown_genus],[uncultured Geoarchaeota archaeon],[1448936],2711,2
4,Archaea,Archaea,Candidatus Heimdallarchaeota,unknown_class,unknown_order,[unknown_family],[unknown_genus],"[Candidatus Heimdallarchaeota archaeon, Candid...","[2026747, 2876573, 2876572]",16162785,47
...,...,...,...,...,...,...,...,...,...,...,...
2111,Viruses,Zilligvirae,Taleaviricota,Tokiviricetes,Primavirales,[Tristromaviridae],"[Betatristromavirus, Alphatristromavirus]","[Betatristromavirus TTV1, Alphatristromavirus ...","[10479, 1805492, 2730621]",54302,6
2112,Viruses,unknown_kingdom,unknown_phylum,Naldaviricetes,Lefavirales,"[Baculoviridae, Hytrosaviridae, Nudiviridae]","[Alphabaculovirus, Betabaculovirus, unknown_ge...","[Bombyx mori nucleopolyhedrovirus, Autographa ...","[271108, 46015, 28289, 74320, 10469, 10456, 46...",47177140,2903
2113,Viruses,unknown_kingdom,unknown_phylum,Naldaviricetes,unknown_order,[Nimaviridae],[Whispovirus],"[White spot syndrome virus, Metopaulias depres...","[342409, 92652, 1675544, 2488332, 46615]",9748026,539
2114,Viruses,unknown_kingdom,unknown_phylum,unknown_class,unknown_order,"[Kolmioviridae, Pospiviroidae, unknown_family,...","[Deltavirus, Coleviroid, unknown_genus, Hostuv...","[Hepatitis delta virus, Coleus blumei viroid 1...","[12475, 192025, 12436, 195062, 12893, 12412, 1...",379543353,76215


In [192]:
df_grouped.shape[0]

2116

If we didn't run the step for filling NA values, how many rows are we losing by grouping data, which implicitly drops NAs?

In [ ]:
all_data.shape[0] - all_data.dropna().shape[0]

In [195]:
for col in all_data.columns:
  print(f'NA values in column {col}: ')
  print(all_data[col].isna().sum())

NA values in column superkingdom: 
0
NA values in column kingdom: 
0
NA values in column phylum: 
0
NA values in column class: 
0
NA values in column order: 
0
NA values in column family: 
0
NA values in column genus: 
0
NA values in column species: 
0
NA values in column taxon_id: 
0
NA values in column total_length: 
0
NA values in column num_accessions: 
0
